# 単純ベイズの実装

In [2]:
import numpy as np


class NaiveBayes():
    def __init__(self):
        self.pY_ = None    # p(y)
        self.pXgY_ = None  # p(xi|y)

    def fit(self, X, y):
        
        # 定数を定義
        n_samples = X.shape[0]   # サンプル数
        n_features = X.shape[1]  # 特徴数
        n_classes = 2  # クラス数 (2値)
        n_fvalues = 2  # 特徴数 (2値)

        # 特徴の事例数とクラスラベルの事例数は一致していなければならない
        if n_samples != len(y):
            raise ValueError('Mismatched number of samples.')

        # N[yi=y] の数を数え上げる
        nY = np.zeros(n_classes, dtype=int)
        for i in range(n_samples):
            nY[y[i]] += 1

        # pY_ (p(y_i)) を計算する
        self.pY_ = np.empty(n_classes, dtype=float)
        for i in range(n_classes):
            self.pY_[i] = nY[i] / n_samples

        # N[x_ij=xj, yi=y] の数を数え上げる
        nXY = np.zeros((n_features, n_fvalues, n_classes), dtype=int)
        for i in range(n_samples):
            for j in range(n_features):
                nXY[j, X[i, j], y[i]] += 1

        # pXgY_ (p(xi|y)) を計算する
        self.pXgY_ = np.empty((n_features, n_fvalues, n_classes),dtype=float)
        for j in range(n_features):
            for xi in range(n_fvalues):
                for yi in range(n_classes):
                    self.pXgY_[j, xi, yi] = nXY[j, xi, yi] / nY[yi]

    def predict(self, X):

        n_samples = X.shape[0]
        n_features = X.shape[1]
        y = np.empty(n_samples, dtype=int)

        # 対数同時確率を計算
        for i, xi in enumerate(X):
            logpXY = (np.log(self.pY_) +
                      np.sum(np.log(self.pXgY_[np.arange(n_features), xi, :]), axis=0))

            # クラスを予測
            y[i] = np.argmax(logpXY)

        return y

## NaiveBayesクラスの実装

In [8]:

data = np.genfromtxt('vote_filled.tsv', dtype=int)
X = data[:, :-1]
y = data[:, -1]

bayes=NaiveBayes()
bayes.fit(X,y)

predict_y = bayes.predict(X[:10, :])
print('n T P')
for i in range(10):
    print(i, y[i], predict_y[i])


n T P
0 1 1
1 1 1
2 0 0
3 0 0
4 0 0
5 0 0
6 0 1
7 1 1
8 1 1
9 0 0


## 予測の実行
データセットにはUCI Repositpryの[Congressional Voting Records Data Set](http://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records)を用いる。
これはアメリカ議会での16種の議題に対する投票行動を特徴とし，議員が共和党 (0) と民主党 (1) のいずれであるかがクラスである。

## 参考文献
Toshihiro Kamishima "Machine Learning Meets Python" https://github.com/tkamishima/mlmpy [online accessed: 2020/05/17]